    ## Vector Store and Retrievers

    ##### Vector store and retriervers abstractions 
    abstractions are designed to support retrieval of data from a vector store and other source fro integration of LLM they are important for applications that fetch data to be reasoned over as part of model inference 

    we will cover :
    documents 
    vector store 
    Retrievers 

## Documents 
Langchain implements a Document abstration , which is intended to represent a unit of text and associated metadata , it has two attributes :

- page_content: The text content of the document.
- metadata: A dictionary of metadata associated with the document. source , relation with other documents , etc.

In [5]:
from langchain_core.documents import Document

documents = [
    Document (
    page_content="Dogs are great companions, known for their loyalty and friendliness.",
    metadata={"source": "mammal-pets-doc"},

    ),

    Document (
    page_content="Cats are independent pets that often enjoy their own space.",
    metadata={"source": "mammal-pets-doc"},

    ),

    Document (

    page_content="Parrots are intelligent birds capable of mimicking human speech.",
    metadata={"source": "bird-pets-doc"},

    ),
    Document (
        page_content = "Rabbits are social animals that need plenty of space to hop around.",
        metadata = {"source": "mammal-pets-doc"},
    )

]

## Vectorstore 
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_chroma import Chroma
groq_api_key = os.getenv("GROQ_API_KEY")
os.environ['HF_TOKEN'] = os.getenv("HF_TOKEN")
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-guard-3-8b",
    api_key=groq_api_key,
)

llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x79141694ea70>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x791416984d00>, model_name='llama-guard-3-8b', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [6]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={"device": "cuda"},
)

/home/omkar/anaconda3/envs/mlenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-05-31 12:44:43.877351: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748675684.001061   60378 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748675684.028808   60378 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1748675684.271123   60378 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than o

In [7]:
## Vectorstore
vectorstore = Chroma.from_documents(documents, embeddings)
vectorstore

In [9]:
vectorstore.similarity_search(
    "What are some good pets?",
    k=2,

)

[Document(id='145a8659-10f7-456f-abcf-f54d7ee5490d', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(id='b5f66477-6e68-42dc-b4fb-ab822e702589', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')]

In [10]:
## Async Query 
await vectorstore.asimilarity_search(
    "What are some good pets?",
    k=2,
)

[Document(id='145a8659-10f7-456f-abcf-f54d7ee5490d', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(id='b5f66477-6e68-42dc-b4fb-ab822e702589', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')]

In [11]:
vectorstore.similarity_search_with_score(
    "What are some good pets?",
    k=2,

)

[(Document(id='145a8659-10f7-456f-abcf-f54d7ee5490d', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
  0.9567375183105469),
 (Document(id='b5f66477-6e68-42dc-b4fb-ab822e702589', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
  1.2338745594024658)]

In [12]:
## Retrivers 
## langchain vectorstore objects do not subclass runable and so can not immediately be integrated into langchain expression language chain 

# langchain retriever are runnables so they implement a standard set of methods (sync and async) and are designed to be incorporated in LCEL chains 

# we can create a simple version of this ourselves without subclassing if we choose what method we wish to use to retrieve documents we can create a runnable easily bellow we will build one around the similarity search mehthod :

In [14]:
from typing import List 
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda


retriever = RunnableLambda(vectorstore.similarity_search).bind(k=1)
retriever.batch(["cat", "dog", "parrot"])

[[Document(id='b5f66477-6e68-42dc-b4fb-ab822e702589', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(id='145a8659-10f7-456f-abcf-f54d7ee5490d', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')],
 [Document(id='52bb7393-d876-4ed5-88e8-1f1a9fc96ea3', metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.')]]

Vectorstores implement an as retriever method that will generate a Retriever specifically a vectrostoreReriever . These retrivers include specific search_type and search_kwargs attributes that identify what methods of the underlying vector store to call and hwo to parameterize them for instace we can replicate the above with the following 

In [16]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1},
)

retriever.batch(["cat", "dog", "parrot"])

[[Document(id='b5f66477-6e68-42dc-b4fb-ab822e702589', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(id='145a8659-10f7-456f-abcf-f54d7ee5490d', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')],
 [Document(id='52bb7393-d876-4ed5-88e8-1f1a9fc96ea3', metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.')]]

In [23]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message ="""
Answer this question using the provided context only.

{question}

Context:
{context}

"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        ("human", message),
        ("ai", "Sure, here is the answer to your question."),
    ]
)

rag_chain = {
    "context": retriever, 
    "question": RunnablePassthrough(),
} | prompt | llm

response = rag_chain.invoke( "tell me about dogs" )
print(response)
response.content

content='safe' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 2, 'prompt_tokens': 198, 'total_tokens': 200, 'completion_time': 0.002003453, 'prompt_time': 0.00500896, 'queue_time': 0.24739444900000002, 'total_time': 0.007012413}, 'model_name': 'llama-guard-3-8b', 'system_fingerprint': 'fp_60081290c2', 'finish_reason': 'stop', 'logprobs': None} id='run--6d63630e-9b5a-4f32-a673-e400956cfa96-0' usage_metadata={'input_tokens': 198, 'output_tokens': 2, 'total_tokens': 200}


'safe'